# Setup

In [2]:
# h/t https://www.dremio.com/blog/getting-hands-on-with-polaris-oss-apache-iceberg-and-apache-spark/
import pyspark
from pyspark.sql import SparkSession
import os

## DEFINE SENSITIVE VARIABLES
POLARIS_URI = 'http://polaris:8181/api/catalog'
POLARIS_CATALOG_NAME = 'polariscatalog'
POLARIS_CREDENTIALS = 'root:secret'
POLARIS_SCOPE = 'PRINCIPAL_ROLE:ALL'

In [3]:
conf = (
    pyspark.SparkConf()
        .setAppName('rmoff')
  		#packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.9.1,org.apache.hadoop:hadoop-aws:3.4.0')
  		#SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
  		#Configuring Catalog
        .set('spark.sql.catalog.polaris', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.polaris.warehouse', POLARIS_CATALOG_NAME)
        .set('spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation', 'true')
        .set('spark.sql.catalog.polaris.catalog-impl', 'org.apache.iceberg.rest.RESTCatalog')
        .set('spark.sql.catalog.polaris.uri', POLARIS_URI)
        .set('spark.sql.catalog.polaris.credential', POLARIS_CREDENTIALS)
        .set('spark.sql.catalog.polaris.scope', POLARIS_SCOPE)
        .set('spark.sql.catalog.polaris.token-refresh-enabled', 'true')
        .set('spark.sql.defaultCatalog', 'polaris')
        # S3 Configuration
        .set('spark.hadoop.fs.s3.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
        .set('spark.hadoop.fs.AbstractFileSystem.s3.impl', 'org.apache.hadoop.fs.s3a.S3A')
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

Spark Running


25/07/14 14:37:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
# # Run this for debug of the config
# for item in spark.sparkContext.getConf().getAll():
#     print(f"{item[0]}: {item[1]}")

Expect the following warning the first time that you run something:

```
WARN RESTSessionCatalog: Iceberg REST client is missing the OAuth2 server URI configuration[…]
```

# Create database and table

In [5]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS polaris.rmoff")
spark.sql("SHOW DATABASES IN polaris").show()

25/07/14 14:37:17 WARN RESTSessionCatalog: Iceberg REST client is missing the OAuth2 server URI configuration and defaults to http://polaris:8181/api/catalog/v1/oauth/tokens. This automatic fallback will be removed in a future Iceberg release.It is recommended to configure the OAuth2 endpoint using the 'oauth2-server-uri' property to be prepared. This warning will disappear if the OAuth2 endpoint is explicitly configured. See https://github.com/apache/iceberg/issues/10537


+---------+
|namespace|
+---------+
|    rmoff|
+---------+



In [6]:
%%sql
USE polaris.rmoff;

25/07/14 14:37:18 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [7]:
%%sql
CREATE TABLE
  IF NOT EXISTS customers (
    customer_id BIGINT,
    first_name VARCHAR(255),
    last_name VARCHAR(255),
    email VARCHAR(255)
  );

++
||
++
++

# Insert a row

In [13]:
%%sql
INSERT INTO
  customers (customer_id, first_name, last_name, email)
VALUES
  (1, 'Rey', 'Skywalker', 'rey@rebelscum.org');

++
||
++
++

In [8]:
for i in range(5):
    spark.sql("""
        INSERT INTO customers (customer_id, first_name, last_name, email) 
        VALUES (1, 'Rey', 'Skywalker', 'rey@rebelscum.org')
    """)

# Examine metadata

See https://iceberg.apache.org/docs/latest/spark-queries/#inspecting-tables

In [10]:
%%sql

SELECT COUNT(*) FROM customers;

count(1)
13


In [11]:
%%sql

SELECT COUNT(*), SUM(record_count) FROM polaris.rmoff.customers.files;

count(1),sum(record_count)
6,13


In [24]:
%%sql
SELECT
  file_path,   record_count,   file_size_in_bytes
FROM
  polaris.rmoff.customers.files;

file_path,record_count,file_size_in_bytes
s3://warehouse/rmoff/customers/data/00000-42-783cb8e4-752e-4e2c-b8fa-2dbacc58d82b-0-00001.parquet,1,1367
s3://warehouse/rmoff/customers/data/00001-43-783cb8e4-752e-4e2c-b8fa-2dbacc58d82b-0-00001.parquet,1,1416
s3://warehouse/rmoff/customers/data/00002-44-783cb8e4-752e-4e2c-b8fa-2dbacc58d82b-0-00001.parquet,1,1396
s3://warehouse/rmoff/customers/data/00001-25-20237dcc-1533-4e95-b840-f412f379d717-0-00001.parquet,1,1407
s3://warehouse/rmoff/customers/data/00002-26-20237dcc-1533-4e95-b840-f412f379d717-0-00001.parquet,1,1407
s3://warehouse/rmoff/customers/data/00004-28-20237dcc-1533-4e95-b840-f412f379d717-0-00001.parquet,1,1407
s3://warehouse/rmoff/customers/data/00005-29-20237dcc-1533-4e95-b840-f412f379d717-0-00001.parquet,1,1407
s3://warehouse/rmoff/customers/data/00006-30-20237dcc-1533-4e95-b840-f412f379d717-0-00001.parquet,1,1407
s3://warehouse/rmoff/customers/data/00008-32-20237dcc-1533-4e95-b840-f412f379d717-0-00001.parquet,1,1407
s3://warehouse/rmoff/customers/data/00009-33-20237dcc-1533-4e95-b840-f412f379d717-0-00001.parquet,1,1407


In [20]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Define schema
schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("email", StringType(), True)
])

# Create data for 10 identical rows
data = [(1, 'Rey', 'Skywalker', 'rey@rebelscum.org')] * 10

# Create DataFrame and insert
df = spark.createDataFrame(data, schema)
df.write.mode("append").insertInto("customers")

In [22]:
%%sql
    INSERT INTO customers (customer_id, first_name, last_name, email) 
VALUES (1, 'Rey', 'Skywalker', 'rey@rebelscum.org'),
       (2, 'Hermione', 'Granger', 'hermione@hogwarts.edu'),
       (3, 'Tony', 'Stark', 'tony@starkindustries.com');

++
||
++
++

# Insert a second row

In [ ]:
%%sql
INSERT INTO
  customers (customer_id, first_name, last_name, email)
VALUES
  (2, 'Hermione', 'Granger', 'leviosaaaaa@hogwarts.edu');


# Examine metadata

See https://iceberg.apache.org/docs/latest/spark-queries/#inspecting-tables

In [12]:
%%sql
SELECT
  file_path,
  record_count,
  file_size_in_bytes
FROM
  polaris.rmoff.customers.files;

file_path,record_count,file_size_in_bytes
s3://warehouse/rmoff/customers/data/00000-4-d57e39e8-9aa7-4559-be96-5b111b735210-0-00001.parquet,1,1367
s3://warehouse/rmoff/customers/data/00000-3-d4fb48f7-b6e3-4336-b79f-8402bc40cb06-0-00001.parquet,1,1367
s3://warehouse/rmoff/customers/data/00000-2-6d957619-d041-4c9a-b8ea-2da4dee57956-0-00001.parquet,1,1367
s3://warehouse/rmoff/customers/data/00000-1-710111a1-122d-4129-a650-36c3437e80d9-0-00001.parquet,1,1367
s3://warehouse/rmoff/customers/data/00000-0-17b59969-8e9e-470b-bf5f-4ae76fdfe15f-0-00001.parquet,1,1367
s3://warehouse/rmoff/customers/data/20250714_144010_00012_egk3z-449b15c9-a847-4f1e-adaf-3c099f418cf3.parquet,8,922


# Rewrite data files

In [16]:
%%sql
SELECT COUNT(*) AS data_file_ct, SUM(record_count) AS record_ct FROM polaris.rmoff.customers.files;

data_file_ct,record_ct
1,13


In [13]:
%%sql
CALL polaris.system.rewrite_data_files 
    (table => 'rmoff.customers',
      options => map ('rewrite-all', 'true')
)

25/07/14 14:44:42 WARN RESTSessionCatalog: Iceberg REST client is missing the OAuth2 server URI configuration and defaults to http://polaris:8181/api/catalog/v1/oauth/tokens. This automatic fallback will be removed in a future Iceberg release.It is recommended to configure the OAuth2 endpoint using the 'oauth2-server-uri' property to be prepared. This warning will disappear if the OAuth2 endpoint is explicitly configured. See https://github.com/apache/iceberg/issues/10537


rewritten_data_files_count,added_data_files_count,rewritten_bytes_count,failed_data_files_count
6,1,7757,0


In [ ]:
%%sql
SELECT
  file_path,
  record_count,
  file_size_in_bytes
FROM
  polaris.rmoff.customers.files;

In [ ]:
%%sql
SELECT
  file_path,
  record_count,
  file_size_in_bytes
FROM
  polaris.rmoff.customers.all_data_files;

## Expire snapshots

In [17]:
%%sql

SELECT committed_at, snapshot_id, parent_id, manifest_list FROM polaris.rmoff.customers.snapshots;

committed_at,snapshot_id,parent_id,manifest_list
2025-07-14 14:37:31.531000,2403840741999442414,None,s3://warehouse/rmoff/customers/metadata/snap-2403840741999442414-1-007b3a78-a267-43e5-aee1-1d8f3e5a147d.avro
2025-07-14 14:37:38.926000,3830932525036690208,2403840741999442414,s3://warehouse/rmoff/customers/metadata/snap-3830932525036690208-1-8dab0845-7f6b-4ec4-b417-26c17d993add.avro
2025-07-14 14:37:39.916000,6409867327989167022,3830932525036690208,s3://warehouse/rmoff/customers/metadata/snap-6409867327989167022-1-7b5585b8-fad2-450b-b08f-fa9f277662b6.avro
2025-07-14 14:37:40.921000,325546929694535411,6409867327989167022,s3://warehouse/rmoff/customers/metadata/snap-325546929694535411-1-3a105c0c-dd60-4c91-9042-2d0f7925bbcd.avro
2025-07-14 14:37:41.808000,269825382665437490,325546929694535411,s3://warehouse/rmoff/customers/metadata/snap-269825382665437490-1-d3f8a73b-df91-4b8f-b585-e6f5237c9601.avro
2025-07-14 14:37:42.757000,129306070246549703,269825382665437490,s3://warehouse/rmoff/customers/metadata/snap-129306070246549703-1-5f52fc1a-ac1b-457b-b52f-fa6bed1a233d.avro
2025-07-14 14:40:11.290000,8861050211953882166,129306070246549703,s3://warehouse/rmoff/customers/metadata/snap-8861050211953882166-1-ad6e5d20-590f-40ad-b73a-fca12f412274.avro
2025-07-14 14:42:07.828000,2371922233042001406,8861050211953882166,s3://warehouse/rmoff/customers/metadata/snap-2371922233042001406-1-523344c2-1dbe-4c37-be8c-c56981c0e536.avro
2025-07-14 14:42:08.031000,5882833294520864762,2371922233042001406,s3://warehouse/rmoff/customers/metadata/snap-5882833294520864762-1-1a7f61bc-4c4f-48bd-8017-591b84723e39.avro
2025-07-14 14:42:08.230000,2961764211154500616,5882833294520864762,s3://warehouse/rmoff/customers/metadata/snap-2961764211154500616-1-f971d471-9325-4650-842b-52831cd4dab6.avro


In [ ]:
%%sql

CALL polaris.system.expire_snapshots(table => 'rmoff.customers',
                                                older_than => TIMESTAMP '2025-07-11 12:40:00.000')

In [ ]:
%%sql

SELECT committed_at, snapshot_id, parent_id, manifest_list FROM polaris.rmoff.customers.snapshots;

## Remove orphan files

https://github.com/apache/polaris/issues/214
https://github.com/pavibhai/polaris/commit/09de4c8fefd6894cca14298df389459999cdc58a

In [ ]:
%%sql
    
    CALL polaris.system.remove_orphan_files(table => 'rmoff.customers', 
                                            dry_run => true)

```sql
trino:rmoff> SET SESSION iceberg.remove_orphan_files_min_retention = '10s';
SET SESSION
trino:rmoff> ALTER TABLE customers EXECUTE remove_orphan_files(retention_threshold => '1s');

Query 20250711_135921_00014_4hdju, FAILED, 1 node
Splits: 1 total, 0 done (0.00%)
0.07 [0 rows, 0B] [0 rows/s, 0B/s]

Query 20250711_135921_00014_4hdju failed: Retention specified (1.00s) is shorter than the minimum retention configured in the system (10.00s). Minimum retention can be chan
ged with iceberg.remove-orphan-files.min-retention configuration property or iceberg.remove_orphan_files_min_retention session property

trino:rmoff> ALTER TABLE customers EXECUTE remove_orphan_files(retention_threshold => '15s');
ALTER TABLE EXECUTE
 rows
------
(0 rows)

Query 20250711_135924_00015_4hdju, FINISHED, 1 node
Splits: 1 total, 1 done (100.00%)
0.10 [0 rows, 0B] [0 rows/s, 0B/s]

trino:rmoff>
```

# Time travel

In [ ]:
%%sql

SELECT * FROM customers 

In [ ]:
%%sql

SELECT * FROM customers VERSION AS OF 8348506506333219028

In [ ]:
%%sql

SELECT * from polaris.rmoff.customers.manifests;

In [ ]:
%%sql

SELECT * FROM polaris.rmoff.customers.manifests TIMESTAMP AS OF (NOW() - INTERVAL 30 SECONDS);


## Manifests 

In [ ]:
%%sql

SELECT * from polaris.rmoff.customers.manifests;

## Snapshots

In [ ]:
%%sql

SELECT * from polaris.rmoff.customers.snapshots;

In [ ]:
%%sql

SELECT snapshot_id, manifest_list from polaris.rmoff.customers.snapshots 
    WHERE committed_at > NOW() - INTERVAL 30 MINUTES 
    ORDER BY committed_at ASC LIMIT 1

# metadata

In [ ]:
%%sql

SELECT * from polaris.rmoff.customers.metadata_log_entries;

In [18]:
%%sql

select
    h.made_current_at,
    s.operation,
    h.snapshot_id,
    h.is_current_ancestor
from polaris.rmoff.customers.history h
join polaris.rmoff.customers.snapshots s
  on h.snapshot_id = s.snapshot_id
order by made_current_at;

made_current_at,operation,snapshot_id,is_current_ancestor
2025-07-14 14:37:31.531000,append,2403840741999442414,True
2025-07-14 14:37:38.926000,append,3830932525036690208,True
2025-07-14 14:37:39.916000,append,6409867327989167022,True
2025-07-14 14:37:40.921000,append,325546929694535411,True
2025-07-14 14:37:41.808000,append,269825382665437490,True
2025-07-14 14:37:42.757000,append,129306070246549703,True
2025-07-14 14:40:11.290000,replace,8861050211953882166,True
2025-07-14 14:42:07.828000,append,2371922233042001406,True
2025-07-14 14:42:08.031000,append,5882833294520864762,True
2025-07-14 14:42:08.230000,append,2961764211154500616,True


# Delete

In [ ]:
%%sql

DELETE FROM customers WHERE customer_id=1;

In [ ]:
%%sql
SELECT
    content,
  file_path,
  record_count,
  file_size_in_bytes
FROM
  polaris.rmoff.customers.files;

In [ ]:
%%sql
SELECT
    content,
  file_path,
  record_count,
  file_size_in_bytes
FROM
  polaris.rmoff.customers.all_data_files;

# Update

In [ ]:
%%sql

SELECT * FROM customers

In [ ]:
%%sql

UPDATE customers SET first_name = 'Bob' where first_name = 'Hermione';

# Add column

In [ ]:
%%sql
ALTER TABLE customers DROP COLUMN phone

In [ ]:
%%sql
INSERT INTO customers VALUES (3,'FRED','BLOGGS','fred@hotspot.com','0000 1234 1234');

In [ ]:
%%sql
SELECT * FROM customers;

In [ ]:
%%sql
SELECT * FROM customers TIMESTAMP AS OF (NOW() - INTERVAL 30 MINUTES);

In [ ]:
%%sql
SELECT * FROM customers VERSION AS OF 7334416300196800603;

In [ ]:
%%sql
SELECT * FROM customers;

In [ ]:
%%sql
INSERT INTO customers (customer_id, first_name, last_name, email) 
VALUES        (3, 'Tony', 'Stark', 'tony@starkindustries.com');

In [ ]:
%%sql
DELETE FROM customers;

In [ ]:
%%sql
    CALL polaris.system.rollback_to_snapshot('rmoff.customers', 707189909035517389)